In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torchvision import models
import os
from zipfile import ZipFile
from torchvision.datasets import ImageFolder


In [ ]:
zip_file_path = '/content/archive (10).zip'
extract_folder = 'food11/'
os.makedirs(extract_folder, exist_ok=True)

with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

extracted_files = os.listdir(extract_folder)
extracted_files

['training', 'validation', 'evaluation']

In [ ]:
# Data Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root='food11/training', transform=transform)
validation_dataset = ImageFolder(root='food11/validation', transform=transform)
test_dataset = ImageFolder(root='food11/evaluation', transform=transform)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Choose Pre-trained Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = models.resnet18(pretrained=True)
model2 = models.vgg16(pretrained=True)
model3 = models.densenet121(pretrained=True)
model_list = [model1, model2,model3]
num_epochs=4
print('Device',device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

Device cuda


In [ ]:

train_losses = []
val_losses = []
# Fine-tuning
for model in model_list:
    print('Training model:', model.__class__.__name__)
    if model.__class__.__name__ == 'ResNet':
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 11)

    elif model.__class__.__name__ == 'VGG':
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 11)

    elif model.__class__.__name__ == 'DenseNet':
        model.classifier = nn.Linear(model.classifier.in_features, 11)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
          for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(validation_loader))

        print("Epoch: ", epoch+1, "Train Loss: ", train_loss/len(train_loader), "Validation Loss: " ,val_loss/len(validation_loader))



Training model: ResNet
Epoch:  1 Train Loss:  0.9526820285999273 Validation Loss:  0.5254220370617178
Epoch:  2 Train Loss:  0.3949340412628303 Validation Loss:  0.42354226429705266
Epoch:  3 Train Loss:  0.23502952241396055 Validation Loss:  0.3906392501322208
Epoch:  4 Train Loss:  0.13966463221701217 Validation Loss:  0.37806591557131874
Training model: VGG
Epoch:  1 Train Loss:  0.7722503272846679 Validation Loss:  0.475217977469718
Epoch:  2 Train Loss:  0.3857533823710815 Validation Loss:  0.45657368162991824
Epoch:  3 Train Loss:  0.22905857132711602 Validation Loss:  0.38430024390281353
Epoch:  4 Train Loss:  0.15143215842243635 Validation Loss:  0.4486834237834922
Training model: DenseNet
Epoch:  1 Train Loss:  0.8517075306871562 Validation Loss:  0.4087427127416487
Epoch:  2 Train Loss:  0.3251675481404687 Validation Loss:  0.3354087009750031
Epoch:  3 Train Loss:  0.18639068297053232 Validation Loss:  0.29840223249737863
Epoch:  4 Train Loss:  0.10000566664998778 Validation 

In [7]:
# Evaluation
for model in model_list:
    print('Testing model:', model.__class__.__name__)
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Testing model: ResNet
Accuracy of the network on the test images: 90 %
Testing model: VGG
Accuracy of the network on the test images: 89 %
Testing model: DenseNet
Accuracy of the network on the test images: 92 %
